In [22]:
import os
import pandas
import requests

#Caminho dos arquivos RAW
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
filename = os.path.join(raw_data_path, 'dados.csv')

url_csv = "https://opendata.arcgis.com/datasets/b54234c151aa4d01b488dc12aafd5574_0.csv?outSR=4326"

#save the file
with open(filename, 'wb') as f, \
    requests.get(url_csv, stream=True) as r:
    for line in r.iter_lines():
        f.write(line)
#replace colon for new line in file
with open(filename, 'r') as file:
    filedata = file.read()
    filedata = filedata.replace(',', '\n')
with open(filename, 'w') as file:
    file.write(filedata)

columns = ["dt_notific", "dt_inicio_sintomas", "bairro_resid", "ap_resid","evolucao", "dt_obito", "CEP", "dt_atualizacao","manter"]

df = pandas.read_csv(filename, skiprows=[0,1], delimiter=';',names = columns, error_bad_lines=False)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56674 entries, 0 to 56673
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   dt_notific          56674 non-null  object 
 1   dt_inicio_sintomas  56633 non-null  object 
 2   bairro_resid        56673 non-null  object 
 3   ap_resid            56673 non-null  float64
 4   evolucao            56673 non-null  object 
 5   dt_obito            5274 non-null   object 
 6   CEP                 56673 non-null  float64
 7   dt_atualizacao      56673 non-null  object 
 8   manter              0 non-null      float64
dtypes: float64(3), object(6)
memory usage: 3.9+ MB


In [23]:
df.head()

,dt_notific,dt_inicio_sintomas,bairro_resid,ap_resid,evolucao,dt_obito,CEP,dt_atualizacao,manter
0,"0""27/03/2020",19/03/2020,CASCADURA,3.3,recuperado,NaN,21311060.0,"20/07/2020""",NaN
1,"1""27/03/2020",21/03/2020,ANIL,4.0,recuperado,NaN,22765320.0,"20/07/2020""",NaN
2,"2""27/03/2020",18/03/2020,MADUREIRA,3.3,recuperado,NaN,21360202.0,"20/07/2020""",NaN
3,"3""27/03/2020",23/03/2020,COSTA BARROS,3.3,recuperado,NaN,21655820.0,"20/07/2020""",NaN
4,"4""27/03/2020",21/03/2020,COLEGIO,3.3,recuperado,NaN,21545280.0,"20/07/2020""",NaN


In [24]:
#mapeando zonas de acordo com ap
def GetZona(ap):
    ret = ""
    if ap == 1.0:
        ret = "Centro"
    elif ap == 2.2:
        ret = "Zona Norte" 
    elif ap == 3.2:
        ret = "Zona Norte" 
    elif ap == 3.3:
        ret = "Zona Norte" 
    elif ap == 3.1:
        ret = "Zona Norte" 
    elif ap == 2.1:
        ret = "Zona Sul"
    else:
        ret = "Zona Oeste" #ap >= 4
    return ret

df['zona'] = df['ap_resid'].map(lambda x: GetZona(x))

In [37]:
#selected zones
centro = len(df[df.zona == 'Centro'])
norte = len(df[df.zona == 'Zona Norte'])
sul = len(df[df.zona == 'Zona Sul'])
oeste = len(df[df.zona == 'Zona Oeste'])

#selected neighborhoods

ipanema = len(df[df.bairro_resid == 'IPANEMA'])
leblon = len(df[df.bairro_resid == 'LEBLON'])
botafogo = len(df[df.bairro_resid == 'BOTAFOGO'])
copa = len(df[df.bairro_resid == 'COPACABANA'])

tijuca = len(df[df.bairro_resid == 'TIJUCA'])
vila = len(df[df.bairro_resid == 'VILA ISABEL'])
                
campo_grande = len(df[df.bairro_resid == 'CAMPO GRANDE'])
bangu = len(df[df.bairro_resid == 'BANGU'])
santa_cruz = len(df[df.bairro_resid == 'SANTA CRUZ'])
barra = len(df[df.bairro_resid == 'BARRA DA TIJUCA'])



In [39]:
import plotly.graph_objects as go


#plotting the chart
fig =go.Figure(go.Sunburst(
    labels = ['Zona Oeste','CAMPO GRANDE','BANGU','SANTA CRUZ','BARRA DA TIJUCA','Zona Sul','IPANEMA','LEBLON', 'BOTAFOGO','COPACABANA', 'Zona Norte', 'TIJUCA', 'VILA ISABEL', 'Centro'],
    parents = ['', 'Zona Oeste','Zona Oeste','Zona Oeste','Zona Oeste','', 'Zona Sul', 'Zona Sul','Zona Sul','Zona Sul','','Zona Norte','Zona Norte',''],
    values=[oeste,campo_grande,bangu,santa_cruz,barra,sul,ipanema,leblon,botafogo,copa,norte,tijuca,vila,centro],
    textinfo='label+value'
))

fig.update_layout( title={
        'text': "Distribuição casos Covid19 por Zonas e bairros selecionados",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} #set the title
        )

#to show the figure
fig.show()